In [1]:
!pip install transformers torch

In [4]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from tqdm import tqdm # 진행 상황 확인용

# ⚠️ BERT 관련 라이브러리 (Colab에서 설치 필수)
# !pip install transformers torch

try:
    import torch
    from transformers import AutoTokenizer, AutoModel

    # GPU (CUDA) 사용 여부 확인 및 장치 설정
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"✅ PyTorch Device Set: {DEVICE}")

except ImportError:
    print("FATAL ERROR: torch 또는 transformers 라이브러리가 설치되지 않았습니다.")
    print("Colab에서 !pip install transformers torch 를 실행하세요.")
    exit()

# ----------------------------------------------------
# 1. BERT 임베딩 함수 정의 (사용자 제공 구조 + Device 적용)
# ----------------------------------------------------
def get_bert_embeddings(texts, model_name='bert-base-uncased', max_len=256):

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).to(DEVICE)
    model.eval() # 추론 모드 설정

    EMBEDDING_DIM = 768
    all_embeddings = []

    # 데이터를 배치(Batch) 단위로 처리하여 GPU 메모리 사용 효율을 높입니다.
    BATCH_SIZE = 32

    for i in tqdm(range(0, len(texts), BATCH_SIZE), desc="BERT Embedding"):
        batch_texts = texts[i:i + BATCH_SIZE]

        # 텍스트를 토큰화 및 패딩
        inputs = tokenizer(
            batch_texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_len
        ).to(DEVICE)

        with torch.no_grad():
            outputs = model(**inputs)

        # [CLS] 토큰 벡터 추출 (문장 전체의 의미를 압축한 벡터)
        # 텐서를 CPU로 옮긴 후 NumPy로 변환
        embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        all_embeddings.append(embeddings)

    return np.vstack(all_embeddings)

# ----------------------------------------------------
# 2. 메인 실행 파이프라인
# ----------------------------------------------------

GAME_FILE = ('/content/drive/MyDrive/bigData/game_recommend_system/game_dataset/cleaned_games.csv')

try:
    game_df = pd.read_csv(GAME_FILE)
    print(f"\n✅ Game data loaded successfully: {GAME_FILE}")
except Exception as e:
    print(f"FATAL ERROR: Game data file not found. {e}")
    exit()

# --- 2a. 데이터 전처리 (결측값 처리) ---
game_df['genres'] = game_df['genres'].fillna('')
# 스토리라인이 너무 길 경우 BERT의 max_len(512)을 초과할 수 있으므로,
# 너무 긴 텍스트는 임의의 짧은 텍스트로 대체합니다. (실제 BERT가 truncation 함)
game_df['storyline'] = game_df['storyline'].fillna('No Storyline Available.')


# --- 2b. 스토리라인 BERT 임베딩 생성 (핵심) ---
print("\n--- 1. 스토리라인 BERT 임베딩 시작 (GPU 권장) ---")
storyline_embeddings_bert = get_bert_embeddings(game_df['storyline'].tolist(), max_len=512)
print(f"BERT Storyline Embeddings Shape: {storyline_embeddings_bert.shape}")


# --- 2c. 장르 TF-IDF 임베딩 생성 (보조 특징) ---
print("\n--- 2. 장르 TF-IDF 임베딩 시작 ---")
genre_tfidf = TfidfVectorizer()
genre_matrix = genre_tfidf.fit_transform(game_df['genres']).toarray()
print(f"Genre TF-IDF Matrix Shape: {genre_matrix.shape}")


# --- 2d. 최종 게임 임베딩 하이브리드화 ---
# BERT 임베딩(Dense)과 장르 TF-IDF 벡터(Dense)를 수평으로 결합합니다.
hybrid_game_embeddings = np.hstack([storyline_embeddings_bert, genre_matrix])


# 3. 최종 결과 저장
output_filename = 'hybrid_game_embeddings_BERT_final.npy'
np.save(output_filename, hybrid_game_embeddings)
np.save('final_game_ids.npy', game_df['game_id'].values)


print("\n=======================================================")
print("✨ Step 3: BERT 기반 게임 임베딩 생성 완료 ✨")
print(f"총 임베딩 차원: {hybrid_game_embeddings.shape[1]} 차원")
print(f"저장 파일: {output_filename}")
print("=======================================================")

✅ PyTorch Device Set: cuda

✅ Game data loaded successfully: /content/drive/MyDrive/bigData/game_recommend_system/game_dataset/cleaned_games.csv

--- 1. 스토리라인 BERT 임베딩 시작 (GPU 권장) ---


BERT Embedding: 100%|██████████| 157/157 [02:35<00:00,  1.01it/s]

BERT Storyline Embeddings Shape: (5000, 768)

--- 2. 장르 TF-IDF 임베딩 시작 ---
Genre TF-IDF Matrix Shape: (5000, 21)

✨ Step 3: BERT 기반 게임 임베딩 생성 완료 ✨
총 임베딩 차원: 789 차원
저장 파일: hybrid_game_embeddings_BERT_final.npy


In [7]:
from google.colab import files
files.download("/content/hybrid_game_embeddings_BERT_final.npy")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>